In [164]:
import keras
import pandas as pd
import numpy as np
from keras.layers import *
from keras.activations import *
from keras.callbacks import *
from keras.optimizers import *
from keras.models import *
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
from scipy import signal
import stldecompose

In [165]:
data_orig = pd.read_csv('../data/files/multiple_concatenated_tickers.csv')
data_orig.tail()

,Index,XLE.Open,XLE.High,XLE.Low,XLE.Close,XLE.Volume,XLE.Adjusted,XLB.Open,XLB.High,XLB.Low,...,XLU.Low,XLU.Close,XLU.Volume,XLU.Adjusted,XLRE.Open,XLRE.High,XLRE.Low,XLRE.Close,XLRE.Volume,XLRE.Adjusted
3105,2019-05-06,63.759998,64.519997,63.700001,64.309998,20344700.0,64.309998,55.320000,56.080002,55.320000,...,57.860001,57.980000,13389500.0,57.980000,36.189999,36.340000,35.970001,36.200001,3377200,36.200001
3106,2019-05-07,63.549999,63.799999,62.939999,63.750000,15064500.0,63.750000,55.330002,55.430000,54.590000,...,57.529999,57.799999,12542200.0,57.799999,36.080002,36.139999,35.320000,35.580002,3711800,35.580002
3107,2019-05-08,63.720001,64.279999,63.580002,63.750000,9823700.0,63.750000,54.840000,55.090000,54.599998,...,56.959999,57.009998,28692400.0,57.009998,35.650002,35.919998,35.564999,35.580002,3831500,35.580002
3108,2019-05-09,63.450001,63.990002,62.910000,63.750000,14409500.0,63.750000,54.209999,54.610001,53.779999,...,56.660000,56.939999,16173800.0,56.939999,35.500000,35.740002,35.259998,35.700001,5684500,35.700001
3109,2019-05-10,63.750000,64.209999,62.849998,64.050003,13073400.0,64.050003,54.299999,55.340000,54.070000,...,56.820000,57.959999,25157000.0,57.959999,35.619999,36.200001,35.595001,36.110001,3268400,36.110001


In [166]:
from urllib.request import *
from io import StringIO
import pandas as pd

def df_from_fred(setname):
    # Make GET Request
    response = urlopen(url_for(setname))
    # Read response data
    data = response.read()
    # Convert binary text to utf-8
    text = data.decode('utf-8')
    # Convert text file to pandas dataframe
    TEXTDATA = StringIO(text)
    df = pd.read_csv(TEXTDATA, sep=",")
    return df

def url_for(series):
    """function takes FRED series name as input. For example, GDPC1, or HOUST."""
    return "https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=968&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id="+series+"&scale=left&cosd=1947-01-01&coed=2019-01-01&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Quarterly&fam=avg&fgst=lin&fgsnd=2009-06-01&line_index=1&transformation=lin&vintage_date=2019-05-07&revision_date=2019-05-07&nd=1947-01-01"


In [156]:
# Vital Economic indicators: https://gist.github.com/ngopal/262fce10a7fa4a8467f0c61a13c85dc5
GDPC1 = df_from_fred("GDPC1")
time.sleep(5)
M2 = df_from_fred("M2")
time.sleep(5)
CPALTT01USQ657N = df_from_fred("CPALTT01USQ657N")
time.sleep(5)
PPIACO = df_from_fred("PPIACO")
time.sleep(5)
UMCSENT = df_from_fred("UMCSENT")
time.sleep(5)
PAYEMS = df_from_fred("PAYEMS")
time.sleep(5)
RRSFS = df_from_fred("RRSFS")
time.sleep(5)
HOUST = df_from_fred("HOUST")
time.sleep(5)
ISRATIO = df_from_fred("ISRATIO")
time.sleep(5)
SP500 = df_from_fred("SP500")
time.sleep(5)

# Others
IPMAN = df_from_fred("IPMAN")
time.sleep(5)
MPU9900063 = df_from_fred("MPU9900063")
time.sleep(5)
PCU33443344 = df_from_fred("PCU33443344")
time.sleep(5)
MEHOINUSA672N = df_from_fred("MEHOINUSA672N")
time.sleep(5)
TCMDO = df_from_fred("TCMDO")
time.sleep(5)
FGTCMDODNS = df_from_fred("FGTCMDODNS")
time.sleep(5)
ADSLFAA027N = df_from_fred("ADSLFAA027N")
time.sleep(5)
NCBCMDPMVCE = df_from_fred("NCBCMDPMVCE")
time.sleep(5)
FGCCSAQ027S = df_from_fred("FGCCSAQ027S")
time.sleep(5)
ASTNITA = df_from_fred("ASTNITA")
time.sleep(5)
PCETRIM12M159SFRBDAL = df_from_fred("PCETRIM12M159SFRBDAL")

# IPMAN, MPU9900063, PCU33443344, MEHOINUSA672N, TCMDO, FGTCMDODNS, ADSLFAA027N, NCBCMDPMVCE, FGCCSAQ027S, ASTNITA

In [167]:
data_orig = data_orig\
  .merge(GDPC1, how="left", left_on=data_orig.Index, right_on=GDPC1.DATE).fillna(method="ffill")\
  .drop("key_0", axis=1)\
  .merge(M2, how="left", left_on=data_orig.Index, right_on=M2.DATE).fillna(method="ffill")\
  .drop("key_0", axis=1)\
  .merge(CPALTT01USQ657N, how="left", left_on=data_orig.Index, right_on=CPALTT01USQ657N.DATE).fillna(method="ffill")\
  .drop("key_0", axis=1)\
  .merge(PPIACO, how="left", left_on=data_orig.Index, right_on=PPIACO.DATE).fillna(method="ffill")\
  .drop("key_0", axis=1)\
  .merge(UMCSENT, how="left", left_on=data_orig.Index, right_on=UMCSENT.DATE).fillna(method="ffill")\
  .drop("key_0", axis=1)\
  .merge(PAYEMS, how="left", left_on=data_orig.Index, right_on=PAYEMS.DATE).fillna(method="ffill")\
  .drop("key_0", axis=1)\
  .merge(RRSFS, how="left", left_on=data_orig.Index, right_on=RRSFS.DATE).fillna(method="ffill")\
  .drop("key_0", axis=1)\
  .merge(HOUST, how="left", left_on=data_orig.Index, right_on=HOUST.DATE).fillna(method="ffill")\
  .drop("key_0", axis=1)\
  .merge(ISRATIO, how="left", left_on=data_orig.Index, right_on=ISRATIO.DATE).fillna(method="ffill")\
  .drop("key_0", axis=1)\
  .merge(PCETRIM12M159SFRBDAL, how="left", left_on=data_orig.Index, right_on=PCETRIM12M159SFRBDAL.DATE).fillna(method="ffill")\
  .drop("key_0", axis=1)\
  .merge(ASTNITA, how="left", left_on=data_orig.Index, right_on=ASTNITA.DATE).fillna(method="ffill")\
  .drop("key_0", axis=1)\
  .merge(FGCCSAQ027S, how="left", left_on=data_orig.Index, right_on=FGCCSAQ027S.DATE).fillna(method="ffill")\
  .drop("key_0", axis=1)\
  .merge(NCBCMDPMVCE, how="left", left_on=data_orig.Index, right_on=NCBCMDPMVCE.DATE).fillna(method="ffill")\
  .drop("key_0", axis=1)\
  .merge(ADSLFAA027N, how="left", left_on=data_orig.Index, right_on=ADSLFAA027N.DATE).fillna(method="ffill")\
  .drop("key_0", axis=1)\
  .merge(FGTCMDODNS, how="left", left_on=data_orig.Index, right_on=FGTCMDODNS.DATE).fillna(method="ffill")\
  .drop("key_0", axis=1)\
  .merge(TCMDO, how="left", left_on=data_orig.Index, right_on=TCMDO.DATE).fillna(method="ffill")\
  .drop("key_0", axis=1)\
  .merge(MEHOINUSA672N, how="left", left_on=data_orig.Index, right_on=MEHOINUSA672N.DATE).fillna(method="ffill")\
  .drop("key_0", axis=1)\
  .merge(PCU33443344, how="left", left_on=data_orig.Index, right_on=PCU33443344.DATE).fillna(method="ffill")\
  .drop("key_0", axis=1)\
  .merge(MPU9900063, how="left", left_on=data_orig.Index, right_on=MPU9900063.DATE).fillna(method="ffill")\
  .drop("key_0", axis=1)\
  .merge(IPMAN, how="left", left_on=data_orig.Index, right_on=IPMAN.DATE).fillna(method="ffill")\
  .drop("key_0", axis=1)\
  .merge(SP500, how="left", left_on=data_orig.Index, right_on=SP500.DATE).fillna(method="ffill")\
  .fillna(-1)

In [168]:
data_orig.drop(['DATE_y', 'DATE_x', 'key_0', 'DATE'], axis=1, inplace=True)

In [170]:
list(data_orig.columns)

['Index',
 'XLE.Open',
 'XLE.High',
 'XLE.Low',
 'XLE.Close',
 'XLE.Volume',
 'XLE.Adjusted',
 'XLB.Open',
 'XLB.High',
 'XLB.Low',
 'XLB.Close',
 'XLB.Volume',
 'XLB.Adjusted',
 'XLI.Open',
 'XLI.High',
 'XLI.Low',
 'XLI.Close',
 'XLI.Volume',
 'XLI.Adjusted',
 'XLY.Open',
 'XLY.High',
 'XLY.Low',
 'XLY.Close',
 'XLY.Volume',
 'XLY.Adjusted',
 'XLP.Open',
 'XLP.High',
 'XLP.Low',
 'XLP.Close',
 'XLP.Volume',
 'XLP.Adjusted',
 'XLV.Open',
 'XLV.High',
 'XLV.Low',
 'XLV.Close',
 'XLV.Volume',
 'XLV.Adjusted',
 'XLF.Open',
 'XLF.High',
 'XLF.Low',
 'XLF.Close',
 'XLF.Volume',
 'XLF.Adjusted',
 'XLK.Open',
 'XLK.High',
 'XLK.Low',
 'XLK.Close',
 'XLK.Volume',
 'XLK.Adjusted',
 'XTL.Open',
 'XTL.High',
 'XTL.Low',
 'XTL.Close',
 'XTL.Volume',
 'XTL.Adjusted',
 'XLU.Open',
 'XLU.High',
 'XLU.Low',
 'XLU.Close',
 'XLU.Volume',
 'XLU.Adjusted',
 'XLRE.Open',
 'XLRE.High',
 'XLRE.Low',
 'XLRE.Close',
 'XLRE.Volume',
 'XLRE.Adjusted',
 'GDPC1',
 'M2',
 'CPALTT01USQ657N',
 'PPIACO',
 'UMCSENT',


In [171]:
ticker_lookup = dict([(i[1].split('.')[0], int(i[0])) for i in enumerate(list(data_orig.columns)) if 'Open' in i[1]])
ticker_lookup

{'XLB': 7,
 'XLE': 1,
 'XLF': 37,
 'XLI': 13,
 'XLK': 43,
 'XLP': 25,
 'XLRE': 61,
 'XLU': 55,
 'XLV': 31,
 'XLY': 19,
 'XTL': 49}

In [172]:
CHOSENTICKER = 'QQQ'

In [175]:
days_lookback = 1
theta = 0.05 # This needs to go between pct_change and argmax apply
# pct_df = data_orig.iloc[:,list(ticker_lookup.values())].pct_change(days_lookback).apply(lambda y: y.argmax(), axis=1).fillna("GSPC.Open")
pct_df = data_orig.iloc[:,list(ticker_lookup.values())].pct_change(days_lookback).apply(lambda y: y.argmax(), axis=1).fillna("XLV.Open")

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.
  after removing the cwd from sys.path.


In [176]:
inv_map = {v+'.Open': k for k, v in dict(enumerate(list(ticker_lookup.keys()))).items()}
pct_df.map(lambda x: inv_map[x])

0        5
1        9
2        7
3        0
4        2
5        4
6        7
7        5
8        1
9        2
10       0
11       3
12       0
13       4
14       2
15       1
16       9
17       4
18       2
19       0
20       2
21       0
22       2
23       9
24       7
25       3
26       0
27       9
28       1
29       2
        ..
3080     0
3081     1
3082     6
3083     1
3084     8
3085     2
3086     0
3087     4
3088     8
3089     8
3090     2
3091     8
3092     5
3093     6
3094     4
3095     2
3096     0
3097     5
3098     7
3099     9
3100     6
3101     0
3102     9
3103    10
3104     3
3105     9
3106     5
3107     0
3108     4
3109     0
Length: 3110, dtype: int64

In [177]:
# De-trend data
data_orig_detrended = signal.detrend(data_orig.iloc[:,1:])

scaler = MinMaxScaler(feature_range=(0, 1))
# data_mat = scaler.fit_transform(data_orig.iloc[:,1:])
data_mat = scaler.fit_transform(data_orig_detrended)

In [178]:
seq_len = 180 # days to use for prediction
data = np.array((data_mat))

In [179]:
sequence_length = seq_len + 1
result = []
for index in range(len(data) - sequence_length):
    result.append(data[index: index + sequence_length])

result = np.array(result)

row = round(0.9 * result.shape[0])
train = result[:int(row), :] # Create training set

In [180]:
# np.random.shuffle(train)
x_train = result[:int(row), :]
y_train = to_categorical(pct_df.map(lambda x: inv_map[x]))[:int(row)]
x_test = result[int(row):, :]
y_test = to_categorical(pct_df.map(lambda x: inv_map[x]))[int(row):]

[x_train, y_train, x_test, y_test]

[array([[[0.90389554, 0.90497316, 0.90606556, ..., 0.98592183,
          0.98558089, 0.98523309],
         [0.90289468, 0.90405911, 0.90523959, ..., 1.        ,
          1.        , 1.        ],
         [0.92175623, 0.92270919, 0.92367526, ..., 0.98024238,
          0.97933002, 0.97839891],
         ...,
         [0.82686532, 0.82721261, 0.8275645 , ..., 0.7918213 ,
          0.7890371 , 0.78619893],
         [0.8797671 , 0.88028202, 0.880804  , ..., 0.86755909,
          0.86516048, 0.86271435],
         [0.89495498, 0.89549314, 0.8960387 , ..., 0.88501263,
          0.88262839, 0.88019656]],
 
        [[0.90289468, 0.90405911, 0.90523959, ..., 1.        ,
          1.        , 1.        ],
         [0.92175623, 0.92270919, 0.92367526, ..., 0.98024238,
          0.97933002, 0.97839891],
         [0.94274806, 0.94354057, 0.94434396, ..., 0.9711683 ,
          0.9695305 , 0.9678589 ],
         ...,
         [0.8797671 , 0.88028202, 0.880804  , ..., 0.86755909,
          0.86516048, 0.

In [181]:
LAYERS = 250
model = Sequential()

model.add(LSTM(
    input_dim=data.shape[1],
    output_dim=LAYERS,
    return_sequences=True))
# model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(LSTM(
    LAYERS,
    return_sequences=False))
model.add(Dropout(0.2))

model.add(Dense(
    output_dim=y_train.shape[1]))
model.add(Activation('softmax'))

start = time.time()
rmsprop = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=rmsprop)
print('compilation time : ', time.time() - start)

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  import sys
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(return_sequences=True, input_shape=(None, 87), units=250)`
  import sys


compilation time :  0.022477149963378906


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=11)`
  app.launch_new_instance()


In [182]:
MODELNAME = 'multiplemodeltest_withoutTrend5'
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('./models/'+MODELNAME+'_best.hdf5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1, epsilon=1e-4, mode='min')

/usr/local/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:1065: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [ ]:
VALIDATIONSIZE = 0.25
EPOCHS = 500
#model = keras.models.load_model('./models/'+MODELNAME+'_best.hdf5') 
history = model.fit(
    x_train,
    y_train,
    batch_size=512,
    nb_epoch=EPOCHS,
    validation_split=VALIDATIONSIZE,
    callbacks = [reduce_lr_loss, earlyStopping, mcp_save])

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  # Remove the CWD from sys.path while we load stuff.


Train on 1977 samples, validate on 659 samples
Epoch 1/500
1977/1977 [==============================] - 54s 27ms/step - loss: 3.0718 - val_loss: 3.2157
Epoch 2/500
1977/1977 [==============================] - 42s 21ms/step - loss: 2.9962 - val_loss: 2.8025
Epoch 3/500
1977/1977 [==============================] - 42s 21ms/step - loss: 2.5113 - val_loss: 2.8656
Epoch 4/500
1977/1977 [==============================] - 42s 21ms/step - loss: 2.4116 - val_loss: 2.6719
Epoch 5/500


In [ ]:
best_model = keras.models.load_model('./models/'+MODELNAME+'_best.hdf5') 

In [ ]:
# 5 refers to days of data. 5 days worth. each 1 row has 36 features
days = 5

In [ ]:
from collections import defaultdict
dd = defaultdict(list)
qq = defaultdict(list)
for d in range(days):
    li = []
    for k in sorted(zip(list(ticker_lookup.keys()), best_model.predict(np.reshape(data[-days:], (days, 1, data.shape[1])))[d]), key = lambda x: x[1], reverse=True):
        dd[k[0]].append(k[1])
        li.append(k[0])
    qq[d] = li

In [ ]:
dd

In [ ]:
[qq[i][:10] for i in range(days)]

In [ ]:
lookup_map = {v:k for k, v in inv_map.items()}
# model.predict(np.reshape(data[-days:], (days, 1, data.shape[1])))[0].argmax()
[lookup_map[best_model.predict(np.reshape(data[-days:], (days, 1, data.shape[1])))[i].argmax()] for i in range(5)]

In [ ]:
plt.plot([k for k in best_model.predict(np.reshape(data[-5:], (5, 1, data.shape[1])))])
plt.show()